<a href="https://colab.research.google.com/github/debayant9/Churn-Detection-using-ANN/blob/main/customer_churn_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
dataset = pd.read_csv("/Churn_Modelling.csv")
dataset.head(20)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [6]:
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [7]:
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X.Gender, drop_first=True)

In [8]:
X = pd.concat([X, geography, gender], axis=1)
X = X.drop(['Geography', 'Gender'], axis=1)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [10]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding, BatchNormalization, Input

In [11]:
def create_model(layers, activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i==0:
      model.add(Dense(units=nodes, input_dim=X_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
       model.add(Dense(nodes))
       model.add(Activation(activation))
       model.add(Dropout(0.3))
             
    model.add(Dense(units = 1, kernel_initializer='glorot_uniform', 
                    activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [12]:
model = KerasClassifier(build_fn=create_model)

In [13]:
layers = [[20], [40, 20], [40, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size=[128, 256], epochs=[30])
grid = GridSearchCV(model, param_grid=param_grid, cv=5)

In [14]:
grid_result = grid.fit(X_train, y_train)

Epoch 1/30
50/50 [==============================] - 14s 2ms/step - loss: 0.5919 - accuracy: 0.7120
Epoch 2/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5076 - accuracy: 0.7944
Epoch 3/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4938 - accuracy: 0.7904
Epoch 4/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4832 - accuracy: 0.7929
Epoch 5/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4686 - accuracy: 0.8020
Epoch 6/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4671 - accuracy: 0.7977
Epoch 7/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4652 - accuracy: 0.8040
Epoch 8/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4567 - accuracy: 0.8036
Epoch 9/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.8016
Epoch 10/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4454 - accuracy: 0.8059
Epoch 11/30
50/50 

In [15]:
print(grid.best_params_)
print(grid.best_score_)

{'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]}
0.8537500023841857


In [18]:
pred_y = grid.predict(X_test)
y_pred = pred_y > 0.5

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [22]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)


from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
score=accuracy_score(y_test,y_pred)

print(score)
fpr, tpr, threshold = roc_curve(y_test, y_pred)
auc_sc = auc(fpr, tpr)
print(auc_sc)

[[1546   49]
 [ 224  181]]
0.8635
0.7080962885560588
